# Ejercicio 1

In [1]:
from random import random, randint
from math import log
# 1

def algo_x(p):
    def simy():
        return randint(0, 3)
    
    def px(i):
        try:
            return p[i]
        except IndexError:
            return 0

    while True:
        y = simy()
        u = random()
        if u < px(y) / (0.35):
            return y

# -- Codigo usado para verificar -------------------

# def sim_esp_var(sim, n):
#     esp  = sum(sim() for _ in range(0, n))/n
#     esp2 = sum(sim()**2 for _ in range(0, n))/n

#     var = esp2 - esp**2

#     return esp, var

# p = [0.13, .22, .35,.3]
# esp, _ = sim_esp_var(lambda : algo_x(p), 10_000)

# p0 = 0
# p1 = 0
# p2 = 0
# p3 = 0
# n = 10_000
# for _ in range(10_000):
#     y = algo_x(p)

#     p0 += int(y == 0) / n
#     p1 += int(y == 1) / n
#     p2 += int(y == 2) / n
#     p3 += int(y == 3) / n
# print([p0, p1, p2, p3])



# Ejercicio 2

In [2]:

# 2

def F_1(u):
    if 0 <= u < 2/3:
        return (3/2 * u)**(2/3)
    else:
        return 3 * (u-1/3)

def ejercicio2():
    u = random()

    return F_1(u)

n = 10_000
cgt1 = 0
for _ in range(n):
    cgt1 += int(ejercicio2() > 1)

print(f"Probabilidad estimada de X > 1 = {cgt1/n}")

Probabilidad estimada de X > 1 = 0.3466


# Ejercicio 3

In [3]:

def eventosPoissonNoHomogeneoOptimizado(lamda, interv, lamda_t, T):
    assert len(lamda) == len(interv)
    j = 0 #recorre subintervalos.
    t = -log ( 1 - random() ) / lamda[j]
    eventos = []
    while t <= T:
        if t <= interv[j]:
            V = random()
            if V < lamda_t(t) / lamda[j]:
                eventos.append(t)
            t += -log(1 - random()) / lamda[j]
        else: #t > interv[j]
            t = interv[j] + (t - interv[j]) * lamda[j] / lamda[j + 1]
            j = min(j + 1, len(interv)-1)
    return len(eventos), eventos

def hot_dog():
    interv = [1, 2, 6, 8, 9]
    T = 9
    lamda = [10, 15, 20, 18, 14]

    def lamda_t(t):
        if 0 <= t < 3:
            return 5 + 5 * t
        elif 3 <= t <= 5:
            return 20
        elif 5 < t <= 9:
            return 30-2*t

    nt, _= eventosPoissonNoHomogeneoOptimizado(lamda, interv, lamda_t, T)

    return nt

def sim_esp_var(sim, n):
    esp  = sum(sim() for _ in range(0, n))/n
    esp2 = sum(sim()**2 for _ in range(0, n))/n

    var = esp2 - esp**2

    return esp, var
esp, _ = sim_esp_var(hot_dog, 10_000)
print(f"Numero esperado estimado: {esp}")

Numero esperado estimado: 141.401


# Ejercicio 4


In [4]:
from typing import List, Tuple
import math
from functools import reduce

def montecarlo_multi(g, intervals: List[Tuple[float, float]], n: int):
    """
    Calcula una integral múltiple
    """

    assert g.__code__.co_argcount == len(intervals)

    # obtener a partir de los x_i los u_i a partir de cambios de variable
    # el cambio de variable depende de si los limites es a, b ambos finitos
    # o si son a = 0 y b = infinito
    def aux(): 

        factors = []
        args = []

        for a, b in intervals:
            x = random()
            u = (x * (b - a) + a) if b is not math.inf else 1/x - 1
            p = (b - a) if b is not math.inf else 1/x**2
            args.append(u)
            factors.append(p)
        
        return [reversed(args), factors]

    # generador de samples para obtener el promedio
    def gen_func():
        for _ in range(0, n):

            args, prods = aux()

            prod = reduce(lambda x, y : x*y, prods)

            yield g(*args) * prod

    gen = gen_func()

    # devolver el promedio de la muestra generada
    return (sum(gen))/n

def g(x, y):
    if x**2 + (y - abs(x)**(3/2))**2 <= 1:
        return 1
    else:
        return 0
def area(n):
    return montecarlo_multi(g, [(-1.5, 1.5), (-1.5, 1.5)], n)

print(f"El area estimada de la curva es {round(area(100_000), 6)}")

El area estimada de la curva es 3.14334
